# MLflowによるノートブックでのモデルトレーニングの追跡

ノートブックでMLflowを使用すると、トレーニングしたモデルを追跡できます。このノートブックは Azure Machine Learning のコンピュートインスタンスで実行するので、MLflow をセットアップする必要はありません。

いくつかのデータを準備し、糖尿病を予測するモデルをトレーニングします。オートロギングとカスタムロギングを使用して、ノートブックで MLflow を使用する方法を探ります。

## 始める前に

このノートブックのコードを実行するには、**azure-ai-ml**パッケージの最新版が必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## ワークスペースに接続する

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [2]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## MLflow の設定
Azure Machine Learning studioのコンピュートインスタンス上でこのノートブックを実行しているので、MLflowを設定する必要はない。

ただし、必要なライブラリがインストールされていることは確認しておきましょう。

> 注**：
> もし **mlflow** ライブラリがインストールされていない場合は、`pip install mlflow` を実行してインストールしてください。

In [3]:
pip show mlflow

Name: mlflow
Version: 2.17.2
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: 
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.
        
                                        Apache License
                                   Version 2.0, January 2004
                                http://www.apache.org/licenses/
        
           TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION
        
           1. Definitions.
        
              "License" shall mean the terms and conditions for use, reproduction,
              and distribution as defined by Sections 1 through 9 of this document.
        
              "Licensor" shall mean the copyright owner or entity authorized by
              the copyright owner that is granting the License.
        
              "Legal Entity" shall mean the union of the acting entity and all
              other entities that control, are controlled by, 

In [4]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


## Prepare the data

糖尿病分類モデルをトレーニングします。学習データは**data**フォルダに**diabetes.csv**として格納されています。
まず、データを読んでみましょう：

In [5]:
import pandas as pd

print("Reading data...")
df = pd.read_csv('./data/diabetes.csv')
df.head()

Reading data...


PatientID  Pregnancies  PlasmaGlucose  DiastolicBloodPressure  \
0    1354778            0            171                      80   
1    1147438            8             92                      93   
2    1640031            7            115                      47   
3    1883350            9            103                      78   
4    1424119            1             85                      59   

   TricepsThickness  SerumInsulin        BMI  DiabetesPedigree  Age  Diabetic  
0                34            23  43.509726          1.213191   21         0  
1                47            36  21.240576          0.158365   23         0  
2                52            35  41.511523          0.079019   23         0  
3                25           304  29.582192          1.282870   43         1  
4                27            35  42.604536          0.549542   22         0

Next, you'll split the data into features and the label (Diabetes):

In [6]:
print("Splitting data...")
X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

Splitting data...


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

モデルの結果を追跡するもう1つの理由は、別の推定量をテストするときです。これまで学習したモデルはすべてロジスティック回帰推定を用いています。

次のセルを実行して、決定木を用いてモデルを訓練し、他の実行と比較して精度が高いかどうかを確認します。

In [14]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

with mlflow.start_run():
    model = DecisionTreeClassifier().fit(X_train, y_train)

    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    mlflow.log_param("estimator", "DecisionTreeClassifier")
    mlflow.log_metric("Accuracy", acc)

2024/11/09 04:49:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run plum_duck_ycd6smzy at: https://westus.api.azureml.ms/mlflow/v2.0/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/#/experiments/868412a1-1e72-44ae-a150-3e1cebcbdf51/runs/9f94a26d-0d0f-428e-8013-b7079bc00eef.
2024/11/09 04:49:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://westus.api.azureml.ms/mlflow/v2.0/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/#/experiments/868412a1-1e72-44ae-a150-3e1cebcbdf51.


最後に、アーティファクトを記録してみよう。アーチファクトはどのようなファイルでもよい。例えば、ROC曲線をプロットし、そのプロットを画像として保存することができる。その画像をアーティファクトとしてログに記録することができる。

以下のセルを実行して、パラメータ、メトリクス、アーチファクトをログに記録します。

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np

with mlflow.start_run():
    model = DecisionTreeClassifier().fit(X_train, y_train)

    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    # plot ROC curve
    y_scores = model.predict_proba(X_test)

    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")

    mlflow.log_param("estimator", "DecisionTreeClassifier")
    mlflow.log_metric("Accuracy", acc)
    mlflow.log_artifact("ROC-Curve.png")

Azure Machine LearningスタジオのJobsページでモデルの結果を確認する。

- パラメータは、**Overview** タブの **Params** の下にあります。
- メトリクスは、**Overview** タブの **Metrics** と、**Metrics** タブにあります。
- 成果物は、**Outputs + logs** タブにあります。
